In [1]:
import torch
import kenlm
import ctcdecode
import pickle
import numpy as np

from datasets import load_from_disk, load_dataset, load_metric, Dataset, concatenate_datasets

In [83]:
from ctcdecode import CTCBeamDecoder

labels = ['o', 'm', '\u00f6', 'v', 'p', 'y', 'z', 'f', 'd', 'j', 'i', 't', 'r', '\u00e4', 'n', 'w', 'h', 'l', 'u', 'a', 'x', 's', 'b', 'c', '\u00df', '\u00fc', 'e', 'g', 'q', ' ', 'k', '_', '_']

decoder = CTCBeamDecoder(
    labels,
    model_path='lm_data/train_utf.arpa',
    alpha=0.5,
    beta=0,
    cutoff_top_n=40,
    cutoff_prob=np.log(0.000001),
    beam_width=128,
    num_processes=2,
    blank_id=32,
    log_probs_input=True
)

In [82]:
from ctcdecode import CTCBeamDecoder

#labels = ['o', 'm', '\u00f6', 'v', 'p', 'y', 'z', 'f', 'd', 'j', 'i', 't', 'r', '\u00e4', 'n', 'w', 'h', 'l', 'u', 'a', 'x', 's', 'b', 'c', '\u00df', '\u00fc', 'e', 'g', 'q', ' ', 'k', '_', '_']

vocab = ['o', 'm', 'ö', 'v', 'p', 'y', 'z', 'f', 'd', 'j', 'i', 't', 'r', 'ä', 'n', 'w', 'h', 'l', 'u', 'a', 'x', 's', 'b', 'c', 'ß', 'ü', 'e', 'g', 'q', ' ', 'k', '_', '_']


decoder = CTCBeamDecoder(
    labels,
    model_path=None,
    alpha=0.5,
    beta=0,
    cutoff_top_n=40,
    cutoff_prob=np.log(0.000001),
    beam_width=128,
    num_processes=2,
    blank_id=32,
    log_probs_input=True
)

In [100]:
lm_decoder = CTCBeamDecoder(
    labels,
    model_path='lm_data/train_utf.arpa',
    alpha=0.5,
    beta=0,
    cutoff_top_n=100,
    cutoff_prob=np.log(0.0000001),
    beam_width=256,
    num_processes=2,
    blank_id=32,
    log_probs_input=True
)

In [73]:
f = open("lm_data/train.arpa", "r", encoding="ISO-8859-1")

content= f.read()
f.close()

f= open("lm_data/train_utf.arpa", 'w', encoding="utf-8")
f.write(content)
f.close()

print("done")

done


In [3]:
res_51_full_log_2 = load_from_disk("lm_res51/res51_full_log_2")

In [29]:
res_51_full_full = load_from_disk("lm_full_full")

In [101]:
#vocab_dict = {"o": 0, "m": 1, "\u00f6": 2, "v": 3, "p": 4, "y": 5, "z": 6, "f": 7, "d": 8, "j": 9, "i": 10, "t": 11, "r": 12, "\u00e4": 13, "n": 14, "w": 15, "h": 16, "l": 17, "u": 18, "a": 19, "x": 20, "s": 21, "b": 22, "c": 23, "\u00df": 24, "\u00fc": 25, "e": 26, "g": 27, "q": 28, "k": 30, " ": 29, "[UNK]": 31, "[PAD]": 32}

vocab_dict = {'o': 0, 'm': 1, 'ö': 2, 'v': 3, 'p': 4, 'y': 5, 'z': 6, 'f': 7, 'd': 8, 'j': 9, 'i': 10, 't': 11, 'r': 12, 'ä': 13, 'n': 14, 'w': 15, 'h': 16, 'l': 17, 'u': 18, 'a': 19, 'x': 20, 's': 21, 'b': 22, 'c': 23, 'ß': 24, 'ü': 25, 'e': 26, 'g': 27, 'q': 28, 'k': 30, ' ': 29, '[UNK]': 31, '[PAD]': 32}
inv_map = {v: k for k, v in vocab_dict.items()}

for i in range(20):
    logs = res_51_full_log_2[i]["lm_raw_2"]
    beam_results, beam_scores, timesteps, out_len = lm_decoder.decode(torch.tensor(logs))

    res = ""
    for n in beam_results[0][0][:out_len[0][0]]:
        res = res + inv_map[int(n)]

    pred_text = res_51_full_log_2[i]['pred_str']
    tar_text = res_51_full_log_2[i]['target_text']
    old_lm = res_51_full_full[i]['lm_str']
    print(i)
    print("Prd: ",pred_text)
    print("Tar: ",tar_text)
    print("Old: ",old_lm)
    print("New: ",res)

0
Prd:  zieht euch bitte draußen die schuhe aus
Tar:  zieht euch bitte draußen die schuhe aus 
Old:  zieht euch bitte draußen die schuhe aus 
New:  zieht euch bitte draußen die schuhe aus 
1
Prd:  des komtikolnegabentert
Tar:  es kommt zum showdown in gstaad 
Old:  des kontingentes 
New:  das komitee gadenstedt 
2
Prd:  ihre fortestrecken erschienen mit molemagazine wie der wog at das basarwaryclar
Tar:  ihre fotostrecken erschienen in modemagazinen wie der vogue harpers bazaar und marie claire 
Old:  ihre forte strecken erschienen mit modemagazin wie der vogtes basar barrique 
New:  ihre forte strecken erschienen mit modemagazin wie der vogtes basar war regler 
3
Prd:  verlibert eine auch für monachen ungewöhnlich lange titelliste
Tar:  felipe hat eine auch für monarchen ungewöhnlich lange titelliste 
Old:  verliert eine auch für monarchen ungewöhnlich lange titelliste 
New:  verliert eine auch für monarchen ungewöhnlich lange titelliste 
4
Prd:  er wurde zu ehren des reichskanzlers o

In [102]:
def infer_lm(batch):
    beam_results, beam_scores, timesteps, out_len = lm_decoder.decode(torch.tensor(batch["lm_raw_2"]))
    res = ""
    for n in beam_results[0][0][:out_len[0][0]]:
        res = res + inv_map[int(n)]
        
    batch["lm_2_str"] = res
    return batch

In [103]:
res_51_full_log_2 = res_51_full_log_2.map(infer_lm, num_proc=8)

Parameter 'function'=<function infer_lm at 0x7f0488796790> of the transform datasets.arrow_dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Parameter 'function'=<function infer_lm at 0x7f0488796790> of the transform datasets.arrow_dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hash

Parameter 'function'=<function infer_lm at 0x7f0488796790> of the transform datasets.arrow_dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [104]:
wer_metric = load_metric("wer")
print("Full Test WER: {:.3f}".format(wer_metric.compute(predictions=res_51_full_log_2["pred_str"], references=res_51_full_log_2["target_text"])))

Full Test WER: 0.147


In [105]:
print("Full Test WER: {:.3f}".format(wer_metric.compute(predictions=res_51_full_log_2["lm_2_str"], references=res_51_full_log_2["target_text"])))

Full Test WER: 0.129


In [106]:
print("Full Test WER: {:.3f}".format(wer_metric.compute(predictions=res_51_full_full["lm_str"], references=res_51_full_full["target_text"])))

Full Test WER: 0.132


In [108]:
res_51_full_log_2.save_to_disk("lm_full_full_2")